# 🌐 Azure Translator & OpenAI - Projeto de Tradução de Textos

Este notebook demonstra o uso de dois serviços de tradução da Azure:
- **Azure Translator API**: Tradução rápida e direta de textos
- **Azure OpenAI (GPT-4o-mini)**: Tradução contextual e inteligente

## 📋 Funcionalidades
1. Tradução de textos simples
2. Extração e tradução de conteúdo web
3. Tradução de documentos Word (.docx)
4. Gerenciamento seguro de credenciais (Colab Secrets + Variáveis de Ambiente)

---

## 1️⃣ Instalação de Dependências

Instalamos as bibliotecas necessárias para:
- Fazer requisições HTTP (`requests`)
- Manipular documentos Word (`python-docx`)
- Extrair texto de páginas web (`beautifulsoup4`)
- Integrar com Azure OpenAI (`langchain-openai`)

In [ ]:
!pip install -q requests python-docx beautifulsoup4 langchain-openai

Note: you may need to restart the kernel to use updated packages.


## 2️⃣ Importação de Bibliotecas

Organizamos as importações por categoria para melhor compreensão.

In [ ]:
# Gerenciamento de secrets e variáveis de ambiente
import os
import getpass
from dotenv_vault import load_dotenv
try:
    from google.colab import userdata
    from google.colab.userdata import TimeoutException, SecretNotFoundError, NotebookAccessError
except:
    print(f"ℹ️ Not in Google Colab environment")
    pass

# Requisições HTTP e processamento de dados
import requests
from bs4 import BeautifulSoup

# Manipulação de documentos
from docx import Document

# Integração com Azure OpenAI
from langchain_openai.chat_models.azure import AzureChatOpenAI

# Utilitários
from pydantic import SecretStr
from dataclasses import dataclass
from typing import Optional, Literal
from pathlib import Path

print("✅ Bibliotecas importadas com sucesso!")

## 3️⃣ Sistema de Gerenciamento de Credenciais

### 🔐 Como funciona?

Este sistema busca credenciais seguindo uma ordem de prioridade:
1. **Colab Secrets** (🔑 painel lateral) - Mais seguro
2. **Variáveis de Ambiente** (os.environ) - Persistente na sessão
3. **Input Manual** (getpass) - Quando nenhuma anterior está disponível

### ⚙️ Recursos
- ✅ Detecção automática de conflitos entre fontes
- ✅ Sincronização de valores entre Colab Secrets e Environment Variables
- ✅ Preview mascarado de credenciais para segurança
- ✅ Workflow interativo para resolver conflitos

In [ ]:
@dataclass
class SecretInfo:
    """Informações sobre o secret obtido"""
    value: str
    source: Literal["colab_secret", "env_var", "user_input"] | None
    secret_name: Optional[str] = None
    env_var_name: Optional[str] = None


@dataclass
class AvailableSecret:
    """Informações sobre um secret disponível"""
    source: Literal["colab_secret", "env_var"]
    name: str
    value: str
    preview: str
    length: int


def get_secret_with_fallback(
    secret_name: str,
    env_var_name: str,
    prompt_message: Optional[str] = None,
    force_input: bool = False,
    update_target: Literal["colab", "env", "both", "none"] | str = "none",
    preferred_source: Optional[Literal["colab_secret", "env_var"]] = None
) -> SecretInfo:
    """
    Obtém um secret seguindo a ordem de prioridade:
    1. Colab userdata (secret)
    2. Variável de ambiente
    3. Input do usuário (getpass)
    
    Args:
        secret_name: Nome do secret no Colab
        env_var_name: Nome da variável de ambiente
        prompt_message: Mensagem customizada para o getpass
        force_input: Se True, ignora secrets existentes e força getpass
        update_target: Onde salvar o valor do getpass ("colab", "env", "both", "none")
        preferred_source: Fonte preferencial quando múltiplas existem
    
    Returns:
        SecretInfo com valor e informações sobre a fonte
    """
    value = None
    source = None
    
    if not force_input:
        # Coleta valores disponíveis
        colab_value = None
        env_value = None
        
        # Verifica Colab secret
        if "COLAB_NOTEBOOK_ID" in os.environ:
            #Tenta carregar o arquivo .env se for disponibilizado no Colab
            
            try:
                colab_value = userdata.get(secret_name)
                if colab_value and not colab_value.strip():
                    colab_value = None
            except (TimeoutException, SecretNotFoundError, NotebookAccessError):
                colab_value = None

            
        
        load_dotenv(override=True)
        # Verifica variável de ambiente
        env_value = os.environ.get(env_var_name)
        if env_value and not env_value.strip():
            env_value = None
        
        # Determina qual usar baseado em preferred_source ou prioridade padrão
        if preferred_source == "colab_secret" and colab_value:
            value = colab_value
            source = "colab_secret"
            print(f"✓ Secret obtido do Colab Secret: '{secret_name}'")
        elif preferred_source == "env_var" and env_value:
            value = env_value
            source = "env_var"
            print(f"✓ Secret obtido da variável de ambiente: '{env_var_name}'")
        elif colab_value:
            value = colab_value
            source = "colab_secret"
            print(f"✓ Secret obtido do Colab Secret: '{secret_name}' (prioridade padrão)")
        elif env_value:
            value = env_value
            source = "env_var"
            print(f"✓ Secret obtido da variável de ambiente: '{env_var_name}'")
    
    # Input do usuário se necessário
    if not value or force_input:
        default_prompt = f"Enter your {env_var_name}: "
        value = getpass.getpass(prompt_message or default_prompt)
        source = "user_input"
        
        # Atualizar variáveis conforme solicitado
        if update_target in ["env", "both"]:
            os.environ[env_var_name] = value
            print(f"✓ Variável de ambiente '{env_var_name}' atualizada")
        
        if update_target in ["colab", "both"]:
            if "COLAB_NOTEBOOK_ID" in os.environ:
                print(f"ℹ️ Para atualizar o Colab Secret '{secret_name}', use a interface do Colab (🔑 no painel lateral)")
            else:
                print(f"⚠️ Não está em ambiente Colab, impossível atualizar secret")
        
        if update_target == "none":
            print(f"ℹ️ Valor obtido mas não salvo permanentemente")
    
    # Se um valor foi obtido de uma fonte e update_target está definido, sincroniza
    elif source and update_target != "none":
        if update_target in ["env", "both"] and source == "colab_secret":
            os.environ[env_var_name] = value
            print(f"✓ Variável de ambiente '{env_var_name}' sincronizada com Colab Secret")
        
        if update_target in ["colab", "both"] and source == "env_var":
            if "COLAB_NOTEBOOK_ID" in os.environ:
                print(f"ℹ️ Para sincronizar o Colab Secret '{secret_name}' com env_var, atualize manualmente (🔑 no painel lateral)")
            else:
                print(f"⚠️ Não está em ambiente Colab")
    
    if not value or not value.strip():
        raise ValueError(f"❌ Não foi possível obter o valor para '{secret_name}' / '{env_var_name}'")
   
    return SecretInfo(
        value=value.strip(),
        source=source,
        secret_name=secret_name,
        env_var_name=env_var_name
    )


def get_available_secrets(secret_name: str, env_var_name: str) -> dict[str, AvailableSecret]:
    """
    Obtém informações sobre secrets disponíveis sem exibi-las
    
    Returns:
        Dict com secrets disponíveis: {"colab_secret": AvailableSecret, "env_var": AvailableSecret}
    """
    available: dict[str, AvailableSecret] = {}
    
    # Verifica Colab secret
    if "COLAB_NOTEBOOK_ID" in os.environ:
        try:
            colab_value = userdata.get(secret_name)
            if colab_value and colab_value.strip():
                available["colab_secret"] = AvailableSecret(
                    source="colab_secret",
                    name=secret_name,
                    value=colab_value,
                    preview=f"{colab_value[:4]}...{colab_value[-4:]}" if len(colab_value) > 8 else "***",
                    length=len(colab_value)
                )
        except:
            pass
    
    # Verifica variável de ambiente
    env_value = os.environ.get(env_var_name)
    if env_value and env_value.strip():
        available["env_var"] = AvailableSecret(
            source="env_var",
            name=env_var_name,
            value=env_value,
            preview=f"{env_value[:4]}...{env_value[-4:]}" if len(env_value) > 8 else "***",
            length=len(env_value)
        )
    
    return available


def show_available_secrets(secret_name: str, env_var_name: str) -> dict[str, AvailableSecret]:
    """
    Mostra quais secrets estão disponíveis e seus valores (mascarados)
    
    Returns:
        Dict com informações sobre disponibilidade
    """
    available = get_available_secrets(secret_name, env_var_name)
    
    # Mostra resultado
    print("\n📋 Secrets disponíveis:")
    print("-" * 50)
    if available:
        for i, (source, secret) in enumerate(available.items(), 1):
            print(f"{i}. {source}: {secret.name} = {secret.preview} (length: {secret.length})")
    else:
        print("❌ Nenhum secret encontrado")
    print("-" * 50)
    
    return available


def check_secrets_match(available: dict[str, AvailableSecret]) -> bool:
    """
    Verifica se todos os secrets disponíveis têm o mesmo valor
    
    Returns:
        True se todos os valores são iguais, False caso contrário
    """
    if len(available) <= 1:
        return True
    
    values = [secret.value for secret in available.values()]
    return all(v == values[0] for v in values)


print("✅ Sistema de gerenciamento de credenciais carregado!")

## 4️⃣ Configuração das Credenciais

### Azure Translator API

Necessário para usar o serviço de tradução da Microsoft Azure.

**Credenciais necessárias:**
- `AZURE_TRANSLATOR_API_KEY` - Chave de autenticação
- `AZURE_TRANSLATOR_REGION` - Região do recurso (ex: brazilsouth, eastus)
- `AZURE_TRANSLATOR_ENDPOINT_TEXT` - Endpoint para tradução de texto
- `AZURE_TRANSLATOR_ENDPOINT_DOC` - Endpoint para tradução de documentos

📚 [Documentação Azure Translator](https://learn.microsoft.com/azure/ai-services/translator/)

In [ ]:
print("\n" + "="*60)
print("🔧 CONFIGURANDO CREDENCIAIS DO AZURE TRANSLATOR")
print("="*60)

try:
    # Endpoint para tradução de texto
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_TRANSLATOR_ENDPOINT_TEXT",
        env_var_name="AZURE_TRANSLATOR_ENDPOINT_TEXT",
        update_target="env"
    )
    tlt_endpoint_url_txt = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    # Endpoint para tradução de documentos
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_TRANSLATOR_ENDPOINT_DOC",
        env_var_name="AZURE_TRANSLATOR_ENDPOINT_DOC",
        update_target="env"
    )
    tlt_endpoint_url_doc = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    # Chave de API
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_TRANSLATOR_API_KEY",
        env_var_name="AZURE_TRANSLATOR_API_KEY",
        update_target="env"
    )
    tlt_subscription_key_1 = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    # Região do Azure
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_TRANSLATOR_REGION",
        env_var_name="AZURE_TRANSLATOR_REGION",
        update_target="env"
    )
    tlt_region = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    print("\n✅ Azure Translator configurado com sucesso!\n")

except ValueError as e:
    print(f"\n❌ Erro ao configurar credenciais: {e}")
    raise

### Azure OpenAI

Necessário para usar modelos GPT via Azure.

**Credenciais necessárias:**
- `AZURE_OPENAI_API_KEY` - Chave de autenticação
- `AZURE_OPENAI_ENDPOINT` - URL do endpoint
- `AZURE_OPENAI_API_VERSION` - Versão da API (ex: 2025-01-01-preview)
- `AZURE_OPENAI_REGION` - Região do recurso
- `AZURE_OPENAI_DEPLOYMENT` - Nome do deployment do modelo

📚 [Documentação Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/)

In [ ]:
print("\n" + "="*60)
print("🔧 CONFIGURANDO CREDENCIAIS DO AZURE OPENAI")
print("="*60)

try:
    # Endpoint do OpenAI
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_OPENAI_ENDPOINT",
        env_var_name="AZURE_OPENAI_ENDPOINT",
        update_target="env"
    )
    oai_endpoint_url = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    # Versão da API
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_OPENAI_API_VERSION",
        env_var_name="AZURE_OPENAI_API_VERSION",
        update_target="env"
    )
    oai_api_version = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    # Chave de API
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_OPENAI_API_KEY",
        env_var_name="AZURE_OPENAI_API_KEY",
        update_target="env"
    )
    oai_subscription_key_1 = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    # Região do Azure
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_OPENAI_REGION",
        env_var_name="AZURE_OPENAI_REGION",
        update_target="env"
    )
    oai_region = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    # Nome do deployment
    secret_info = get_secret_with_fallback(
        secret_name="AZURE_OPENAI_DEPLOYMENT",
        env_var_name="AZURE_OPENAI_DEPLOYMENT",
        update_target="env"
    )
    oai_deployment = SecretStr(secret_info.value)
    print(f"✓ {secret_info.env_var_name} obtida de: {secret_info.source}")

    print("\n✅ Azure OpenAI configurado com sucesso!\n")

except ValueError as e:
    print(f"\n❌ Erro ao configurar credenciais: {e}")
    raise

## 5️⃣ Configurações Padrão

Definimos os idiomas padrão para as traduções.

**Códigos de idioma suportados:**
- `pt-br` - Português (Brasil)
- `en` - Inglês
- `es` - Espanhol
- `fr` - Francês
- `de` - Alemão
- E muitos outros...

🔗 [Lista completa de idiomas suportados](https://learn.microsoft.com/azure/ai-services/translator/language-support)

In [ ]:
# Idiomas padrão para tradução
default_from_language = 'pt-br'
default_target_language = 'en'

print(f"📝 Idioma de origem padrão: {default_from_language}")
print(f"🎯 Idioma de destino padrão: {default_target_language}")

---

# 🛠️ Funções Utilitárias

## 6️⃣ Extração de Texto de Páginas Web

Esta função permite extrair o conteúdo textual de qualquer página web, removendo:
- Tags HTML
- Scripts e estilos
- Espaços em branco excessivos

**Útil para:** Traduzir artigos, notícias, blogs e qualquer conteúdo web.

In [ ]:
def extract_text_from_url(url: str) -> str:
    """
    Extrai texto limpo de uma página web.
    
    Args:
        url: URL da página web
    
    Returns:
        Texto extraído e limpo da página
    
    Raises:
        Exception: Se não conseguir acessar a página
    """
    try:
        # Faz requisição HTTP
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove scripts e estilos
        for script_or_style in soup(["script", "style"]):
            script_or_style.extract()
        
        # Extrai texto
        text = soup.get_text(separator=' ')
        
        # Limpa o texto
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        clean_text = '\n'.join(chunk for chunk in chunks if chunk)
        
        return clean_text
    
    except requests.exceptions.RequestException as e:
        raise Exception(f"Erro ao acessar URL: {e}")


print("✅ Função de extração de texto carregada!")

---

# 🌐 Métodos de Tradução

## 7️⃣ Método 1: Azure OpenAI (GPT-4o-mini)

### Vantagens:
- ✅ Tradução contextual e inteligente
- ✅ Compreende nuances e contexto
- ✅ Melhor para textos criativos ou técnicos

### Desvantagens:
- ⚠️ Mais lento
- ⚠️ Mais caro
- ⚠️ Limitações de tokens

**Quando usar:** Textos que exigem compreensão contextual, adaptação cultural ou estilo específico.

In [ ]:
def openai_translator(
    text: str, 
    from_lang: str = default_from_language, 
    target_lang: str = default_target_language
) -> str:
    """
    Traduz texto usando Azure OpenAI (GPT-4o-mini).
    
    Args:
        text: Texto a ser traduzido
        from_lang: Idioma de origem (padrão: pt-br)
        target_lang: Idioma de destino (padrão: en)
    
    Returns:
        Texto traduzido
    """
    # Inicializa cliente Azure OpenAI
    client = AzureChatOpenAI(
        azure_endpoint=oai_endpoint_url.get_secret_value(),
        api_key=oai_subscription_key_1,
        api_version=oai_api_version.get_secret_value(),
        azure_deployment=oai_deployment.get_secret_value(),
        max_retries=2,
        temperature=0.3  # Baixa temperatura para tradução mais consistente
    )
    
    # Prepara mensagens para o modelo
    messages = [
        ("system", "Você é um tradutor profissional especializado. Traduza o texto preservando o tom, estilo e significado original."),
        ("user", f"Traduza o texto a seguir de '{from_lang}' para '{target_lang}'. Retorne APENAS a tradução, sem explicações adicionais:\n\n{text}")
    ]
    
    # Realiza tradução
    response = client.invoke(messages)
    return response.content


print("✅ Tradutor OpenAI carregado!")

### 🧪 Teste rápido - OpenAI Translator

In [ ]:
# Teste simples
texto_teste = "Devo ficar ou devo ir?"
traducao = openai_translator(texto_teste, target_lang='en')

print(f"📝 Original: {texto_teste}")
print(f"🌐 Tradução: {traducao}")

## 8️⃣ Método 2: Azure Translator API

### Vantagens:
- ✅ Tradução rápida e eficiente
- ✅ Mais barato
- ✅ Sem limitações de tokens
- ✅ Suporta detecção automática de idioma

### Desvantagens:
- ⚠️ Menos contextual
- ⚠️ Pode ser literal demais

**Quando usar:** Tradução de grandes volumes, textos simples ou quando velocidade é prioridade.

In [ ]:
def translator_text(
    text: str, 
    from_lang: str = default_from_language, 
    target_lang: str = default_target_language
) -> dict:
    """
    Traduz texto usando Azure Translator API.
    
    Args:
        text: Texto a ser traduzido
        from_lang: Idioma de origem (padrão: pt-br)
        target_lang: Idioma de destino (padrão: en)
    
    Returns:
        Resposta completa da API incluindo tradução e detecção de idioma
    """
    # Monta URL do endpoint
    path = "/translate"
    constructed_url = tlt_endpoint_url_txt.get_secret_value() + path
    
    # Headers da requisição
    headers = {
        'Ocp-Apim-Subscription-Key': tlt_subscription_key_1.get_secret_value(),
        'Ocp-Apim-Subscription-Region': tlt_region.get_secret_value(),
        'Content-type': 'application/json',
        'X-ClientTraceId': str(os.urandom(16))  # ID único para rastreamento
    }
    
    # Corpo da requisição
    body = [{'text': text}]
    
    # Parâmetros da API
    params = {
        'api-version': '3.0',
        'from': from_lang,
        'to': target_lang
    }
    
    # Realiza requisição
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    
    return response


print("✅ Tradutor Azure API carregado!")

### 🧪 Testes - Azure Translator API

Vamos testar a tradução para diferentes idiomas.

In [ ]:
# Teste 1: Português para Inglês
texto_teste = "Devo ficar ou devo ir?"
resultado = translator_text(texto_teste, target_lang='en')

print("📝 Teste 1: PT-BR → EN")
print(f"Original: {texto_teste}")
print(f"Tradução: {resultado[0]['translations'][0]['text']}")
print(f"Idioma detectado: {resultado[0].get('detectedLanguage', {}).get('language', 'N/A')}")
print("-" * 50)

In [ ]:
# Teste 2: Português para Espanhol
resultado = translator_text(texto_teste, target_lang='es')

print("📝 Teste 2: PT-BR → ES")
print(f"Original: {texto_teste}")
print(f"Tradução: {resultado[0]['translations'][0]['text']}")
print("-" * 50)

In [ ]:
# Teste 3: Português para Francês
resultado = translator_text(texto_teste, target_lang='fr')

print("📝 Teste 3: PT-BR → FR")
print(f"Original: {texto_teste}")
print(f"Tradução: {resultado[0]['translations'][0]['text']}")
print("-" * 50)

## 9️⃣ Tradução de Documentos Word

### Funcionalidade
Traduz documentos Word (.docx) parágrafo por parágrafo, preservando a estrutura do documento.

### Características:
- ✅ Mantém a formatação de parágrafos
- ✅ Salva automaticamente com sufixo do idioma (ex: `_en.docx`)
- ✅ Traduz parágrafo por parágrafo para melhor precisão

### Limitações:
- ⚠️ Não preserva formatação de texto (negrito, itálico, etc.)
- ⚠️ Não traduz tabelas ou imagens
- ⚠️ Não preserva estilos personalizados

In [ ]:
def translator_document_by_text(
    path: str, 
    from_lang: str = default_from_language, 
    target_lang: str = default_target_language
) -> str:
    """
    Traduz um documento Word (.docx) parágrafo por parágrafo.
    
    Args:
        path: Caminho do arquivo .docx original
        from_lang: Idioma de origem (padrão: pt-br)
        target_lang: Idioma de destino (padrão: en)
    
    Returns:
        Caminho do arquivo traduzido
    """
    print(f"\n📄 Iniciando tradução do documento: {path}")
    print(f"🔄 Traduzindo de '{from_lang}' para '{target_lang}'...")
    
    # Carrega documento original
    document = Document(path)
    total_paragraphs = len(document.paragraphs)
    
    print(f"📊 Total de parágrafos: {total_paragraphs}")
    
    # Traduz cada parágrafo
    translated_paragraphs = []
    for i, paragraph in enumerate(document.paragraphs, 1):
        if paragraph.text.strip():  # Ignora parágrafos vazios
            print(f"  ⏳ Traduzindo parágrafo {i}/{total_paragraphs}...", end='\r')
            
            # Traduz o parágrafo
            result = translator_text(paragraph.text, from_lang, target_lang)
            translated_text = result[0]['translations'][0]['text']
            translated_paragraphs.append(translated_text)
        else:
            translated_paragraphs.append("")  # Mantém linha vazia
    
    print(f"\n  ✅ Todos os parágrafos traduzidos!")
    
    # Cria novo documento com texto traduzido
    translated_doc = Document()
    for text in translated_paragraphs:
        translated_doc.add_paragraph(text)
    
    # Salva documento traduzido
    output_path = path.replace(".docx", f"_{target_lang}.docx")
    translated_doc.save(output_path)
    
    print(f"💾 Documento traduzido salvo em: {output_path}")
    return output_path


print("✅ Tradutor de documentos carregado!")

---

# 🎯 Exemplos Práticos

## 📝 Exemplo 1: Tradução de Artigo Web

In [ ]:
# URL do artigo a ser traduzido
url = 'https://dev.to/devteam/congrats-to-the-ai-agents-intensive-course-writing-challenge-winners-1969'

print("🌐 Extraindo texto da página...")
text = extract_text_from_url(url)

print(f"\n📊 Tamanho do texto extraído: {len(text)} caracteres")
print(f"\n📄 Primeiros 300 caracteres:")
print("-" * 60)
print(text[:300] + "...")
print("-" * 60)

print("\n🔄 Traduzindo para inglês usando OpenAI...")
article_translated = openai_translator(text, target_lang='en')

print("\n✅ Tradução concluída!")
print(f"\n📝 Texto traduzido (primeiros 500 caracteres):")
print("-" * 60)
print(article_translated[:500] + "...")
print("-" * 60)

## 📄 Exemplo 2: Tradução de Documento Word

**Nota:** Certifique-se de fazer upload do arquivo .docx antes de executar esta célula.

In [ ]:
# Caminho do arquivo a ser traduzido
# Faça upload do arquivo ou ajuste o caminho conforme necessário
input_file = "/content/exemplo.docx"  # Ajuste conforme necessário

# Verifica se o arquivo existe
import os
if os.path.exists(input_file):
    # Traduz o documento
    output_file = translator_document_by_text(
        input_file, 
        from_lang='pt-br',
        target_lang='en'
    )
    
    print(f"\n✅ Tradução completa!")
    print(f"📥 Arquivo original: {input_file}")
    print(f"📤 Arquivo traduzido: {output_file}")
else:
    print(f"❌ Arquivo não encontrado: {input_file}")
    print("💡 Faça upload do arquivo ou ajuste o caminho.")

---

# 📊 Comparação Entre Métodos

## Teste Comparativo: OpenAI vs Azure Translator

In [ ]:
import time

# Texto para comparação
texto_comparacao = """
A inteligência artificial está revolucionando a forma como trabalhamos e vivemos. 
Desde assistentes virtuais até carros autônomos, a IA está presente em diversos 
aspectos do nosso dia a dia. No entanto, é importante desenvolver essas tecnologias 
de forma ética e responsável.
"""

print("="*70)
print("🔬 TESTE COMPARATIVO: OPENAI vs AZURE TRANSLATOR")
print("="*70)

print(f"\n📝 Texto original ({len(texto_comparacao)} caracteres):")
print("-" * 70)
print(texto_comparacao)
print("-" * 70)

# Teste OpenAI
print("\n🤖 Traduzindo com OpenAI (GPT-4o-mini)...")
start_time = time.time()
traducao_openai = openai_translator(texto_comparacao, target_lang='en')
tempo_openai = time.time() - start_time

print(f"⏱️ Tempo: {tempo_openai:.2f}s")
print(f"📄 Resultado:")
print("-" * 70)
print(traducao_openai)
print("-" * 70)

# Teste Azure Translator
print("\n🌐 Traduzindo com Azure Translator API...")
start_time = time.time()
resultado_azure = translator_text(texto_comparacao, target_lang='en')
traducao_azure = resultado_azure[0]['translations'][0]['text']
tempo_azure = time.time() - start_time

print(f"⏱️ Tempo: {tempo_azure:.2f}s")
print(f"📄 Resultado:")
print("-" * 70)
print(traducao_azure)
print("-" * 70)

# Comparação
print("\n📊 RESUMO COMPARATIVO:")
print("="*70)
print(f"{'Método':<25} {'Tempo':<15} {'Diferença'}")
print("-" * 70)
print(f"{'OpenAI (GPT-4o-mini)':<25} {tempo_openai:.2f}s{'':<12} -")
print(f"{'Azure Translator API':<25} {tempo_azure:.2f}s{'':<12} {tempo_openai/tempo_azure:.1f}x mais rápido")
print("="*70)

---

# 🎓 Conclusão

## Quando usar cada método?

### 🤖 Azure OpenAI (GPT-4o-mini)
**Ideal para:**
- Textos literários ou criativos
- Documentação técnica que requer contexto
- Conteúdo que precisa de adaptação cultural
- Quando a qualidade é mais importante que a velocidade

### 🌐 Azure Translator API
**Ideal para:**
- Grandes volumes de texto
- Tradução em tempo real
- Quando velocidade é prioridade
- Textos simples e diretos

## 🎯 Por Que Essa Diferença?

1. **Especialização**: Azure Translator é um serviço focado **apenas** em tradução - otimizado para isso
2. **Complexidade**: GPT-4o-mini é um modelo **generalista** muito mais complexo e poderoso
3. **Infraestrutura**: LLMs requerem muito mais recursos computacionais
4. **Valor agregado**: Com OpenAI você paga pela capacidade de entender contexto, nuances culturais, adaptar tom, etc.

## 💡 Quando Vale a Pena?

**Azure Translator** → Tradução direta, grandes volumes  
**Azure OpenAI** → Quando você precisa de:
- Adaptação cultural
- Manutenção de tom/estilo
- Tradução criativa
- Contexto complexo

Faz sentido agora? O OpenAI é tipo contratar um tradutor PhD especializado, enquanto o Azure Translator é como usar um dicionário automático super eficiente! 😄

## 💰 Comparação de Custos

### **Azure Translator API** (Serviço Especializado)
- 💵 **Modelo de cobrança**: Por **caracteres** traduzidos
- 📊 **Preço aproximado**: ~$10 USD por **1 milhão de caracteres**
- ⚡ **Eficiência**: Otimizado especificamente para tradução
- 🎯 **Exemplo**: Traduzir 1000 caracteres ≈ $0.01 USD

### **Azure OpenAI - GPT-4o-mini** (Modelo de Linguagem)
- 💵 **Modelo de cobrança**: Por **tokens** (entrada + saída)
- 📊 **Preço aproximado**: 
  - Input: ~$0.15 USD por 1M tokens
  - Output: ~$0.60 USD por 1M tokens
- ⚙️ **Overhead**: Você paga por:
  - System prompt (instruções)
  - User prompt (seu texto)
  - Resposta gerada (tradução)
- 🎯 **Exemplo**: Traduzir 1000 caracteres ≈ $0.15-0.30 USD (muito mais!)

## 📊 Exemplo Prático

Vamos traduzir um texto de **1000 caracteres** (≈ 750 tokens):

```
Texto: 1000 caracteres
```

### Azure Translator:
```
Custo: 1000 caracteres × $10/1M = $0.01
```

### Azure OpenAI (GPT-4o-mini):
```
Input tokens:
- System prompt: ~50 tokens
- Instrução de tradução: ~30 tokens  
- Texto original: ~750 tokens
Total input: ~830 tokens → $0.12

Output tokens:
- Tradução: ~750 tokens → $0.45

Custo total: ~$0.57 (57x mais caro!)
```

## 🆓 Free Tier - A Diferença Fica AINDA MAIOR

### **Azure Translator - Free Tier**
```
✅ 2 MILHÕES de caracteres GRÁTIS por mês
   └─ Permanente (não expira)
   └─ Renova todo mês
   └─ Sem necessidade de cartão de crédito*
```

**Exemplo prático:**
- 2M caracteres = ~4000 páginas de texto!
- Para estudos/projetos pequenos: **100% GRATUITO**
- Você teria que traduzir **66.666 caracteres POR DIA** para esgotar

### **Azure OpenAI - Sem Free Tier Permanente**
```
❌ Não tem free tier contínuo
⚠️ Opções disponíveis:
   └─ Azure Trial: $200 créditos (30 dias)
   └─ Depois: paga por uso desde o primeiro token
```

## 💰 Comparação Real

### Cenário: Projeto de Estudo (50.000 caracteres/mês)

**Azure Translator:**
```
Free Tier: 2.000.000 caracteres/mês
Seu uso:     50.000 caracteres/mês
────────────────────────────────────
Custo:      $0.00 ✅ (4% do limite gratuito)
```

**Azure OpenAI (GPT-4o-mini):**
```
Sem free tier permanente
Custo estimado: ~$3-5/mês ❌
```

### Cenário: Projeto Médio (500.000 caracteres/mês)

**Azure Translator:**
```
Free Tier: 2.000.000 caracteres/mês
Seu uso:     500.000 caracteres/mês
────────────────────────────────────
Custo:      $0.00 ✅ (25% do limite gratuito)
```

**Azure OpenAI:**
```
Custo estimado: ~$30-50/mês ❌
```

### Cenário: Quando você PASSA do free tier (3M caracteres/mês)

**Azure Translator:**
```
Free Tier: 2.000.000 caracteres/mês
Pago:      1.000.000 caracteres/mês
────────────────────────────────────
Custo:      $10.00 💚 (só paga o excedente)
```

**Azure OpenAI:**
```
Custo estimado: ~$180-300/mês ❌
```

## 📊 Tabela Resumida

| Volume/Mês | Azure Translator | Azure OpenAI | Diferença |
|------------|-----------------|--------------|-----------|
| 50K chars | **GRÁTIS** 🎁 | ~$3-5 | ∞ |
| 500K chars | **GRÁTIS** 🎁 | ~$30-50 | ∞ |
| 2M chars | **GRÁTIS** 🎁 | ~$120-200 | ∞ |
| 3M chars | **$10** 💚 | ~$180-300 | **18-30x mais caro** |



## 🎓 Conclusão para Estudos/Projetos Pessoais

**Para o seu projeto de estudo:**

```python
# Se você está aprendendo/experimentando:
usar_azure_translator = True  # ✅ 2M caracteres grátis/mês!

# Se você precisa de tradução contextual ocasional:
usar_openai_para_casos_especiais = True  # 💡 Esporadicamente

# Combinação inteligente:
def escolher_metodo(texto, precisa_contexto):
    if precisa_contexto and len(texto) < 1000:
        return openai_translator(texto)  # Pequeno, vale o custo
    else:
        return translator_text(texto)  # Usa o free tier!
```

## 💡 Dica de Ouro

Para maximizar o free tier:
```python
# ✅ BOM: Traduzir documento de 100 páginas
translator_document_by_text("documento.docx")  # Free tier

# ❌ DESPERDÍCIO: Usar OpenAI para isso
# Custaria ~$50 quando poderia ser grátis!
```

**TL;DR**: Com o free tier do Azure Translator, a diferença não é só de preço - para volumes pequenos/médios, um é **gratuito** e o outro cobra sempre. É tipo comparar Netflix (paga sempre) com YouTube (tem opção grátis)! 🎬📺

Então sim, para o seu projeto de estudos, o Azure Translator é praticamente **imbatível** em custo-benefício! 🚀

*Alguns serviços Azure podem requerer cartão para validação, mas não cobram se você ficar no free tier



## 📚 Recursos Adicionais

- [Azure Translator Documentation](https://learn.microsoft.com/azure/ai-services/translator/)
- [Azure OpenAI Documentation](https://learn.microsoft.com/azure/ai-services/openai/)
- [Language Support](https://learn.microsoft.com/azure/ai-services/translator/language-support)

---

**Desenvolvido para fins educacionais** 📖  
**Última atualização:** Janeiro 2025